In [1]:
import talos
import os
import pandas as pd
import tensorflow as tf
import keras as ks
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("sample_data_intw.csv")

In [12]:
data.head(5)

,label,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
0,0,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,2,...,2,12,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000
1,1,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,1,...,1,12,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000
2,1,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,1,...,1,6,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000
3,1,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,0,...,2,12,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000
4,1,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,7,...,7,42,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333


In [4]:
#seperate categorical and numerical columns
non_num=[]
for i in data.columns:
    if data[i].dtype=="object":
        non_num.append(i)

print(non_num)

['msisdn', 'pcircle', 'pdate']


In [5]:
#drop these columns
data.drop(labels=non_num,axis=1,inplace=True)
#Prepare feature and target sets
x = data.drop(labels=["label"],axis=1)
y = data["label"]
x = talos.utils.rescale_meanzero(x)

In [6]:
#since our data is completely numeric, we can proceed for upsampling
from imblearn.over_sampling import SMOTE
#Initialise smote
smote = SMOTE()
# fit predictor and target variable on smote
x_smote, y_smote = smote.fit_resample(x,y)

print('old shape', x.shape)
print('new shape', x_smote.shape)

old shape (209593, 32)
new shape (366862, 32)


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_smote,y_smote,test_size=0.25,random_state=123)
print("length of x_train {} and y_train {}".format(len(x_train),len(y_train)))
print("length of x_test {} and y_test {}".format(len(x_test),len(y_test)))
# split for validation set
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.20,random_state=123)
print("length of x_train {} and y_train {}".format(len(x_train),len(y_train)))
print("length of x_valid {} and y_valid {}".format(len(x_valid),len(y_valid)))


length of x_train 275146 and y_train 275146
length of x_test 91716 and y_test 91716
length of x_train 220116 and y_train 220116
length of x_valid 55030 and y_valid 55030


In [8]:
model = tf.keras.models.Sequential()
opt = tf.keras.optimizers.Adam(learning_rate=1E-3)

In [9]:
#building architecture
#Adding layers
model.add(ks.layers.Dense(32,activation="relu",name="layer1",input_shape=(32,))) #do this or just add a simple layer but flattened inputs.(2d to 1d)
model.add(ks.layers.Dense(32,name="layer2"))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(32,activation="relu",name="layer3"))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(1,activation="sigmoid",name="output"))#2 classes 


In [10]:
model.compile(loss="binary_crossentropy",
             optimizer=opt,
             metrics=["accuracy"])

In [11]:
#Training and evaluation
history = model.fit(x_train,y_train,epochs=100,
                    validation_data=(x_valid,y_valid),
                    batch_size=32)

Epoch 1/100
6879/6879 [==============================] - 9s 1ms/step - loss: 0.4928 - accuracy: 0.7695 - val_loss: 0.4673 - val_accuracy: 0.7816
Epoch 2/100
6879/6879 [==============================] - 7s 1ms/step - loss: 0.4714 - accuracy: 0.7824 - val_loss: 0.4586 - val_accuracy: 0.7857
Epoch 3/100
6879/6879 [==============================] - 7s 1ms/step - loss: 0.4642 - accuracy: 0.7849 - val_loss: 0.4553 - val_accuracy: 0.7874
Epoch 4/100
6879/6879 [==============================] - 7s 1ms/step - loss: 0.4612 - accuracy: 0.7872 - val_loss: 0.4536 - val_accuracy: 0.7885
Epoch 5/100
6879/6879 [==============================] - 7s 1ms/step - loss: 0.4583 - accuracy: 0.7893 - val_loss: 0.4567 - val_accuracy: 0.7887
Epoch 6/100
6879/6879 [==============================] - 7s 1ms/step - loss: 0.4554 - accuracy: 0.7902 - val_loss: 0.4477 - val_accuracy: 0.7922
Epoch 7/100
6879/6879 [==============================] - 7s 996us/step - loss: 0.4539 - accuracy: 0.7909 - val_loss: 0.4455 - val_

In [13]:
result = model.evaluate(x=x_test,y=y_test)
print("Accuracy Score on Test Data: {}%".format(round(result[1]*100,3)))

2867/2867 [==============================] - 2s 636us/step - loss: 0.4278 - accuracy: 0.8032
Accuracy Score on Test Data: 80.319%
